## Table Of Contents
1. [Initial Setup](#first-bullet)<br>
    * [1.1 Installing Required Libraries](#first-bullet-sub-1)<br>
    * [1.2 Importing required libraries](#first-bullet-sub-2)<br>
2. [Loading Data](#second-bullet)<br>
3. [LLM Prompting](#third-bullet)<br>
    * [3.1 Create model prompt function](#third-bullet-sub-1)<br>
    * [3.2 Prompting LLM](#third-bullet-sub-2)<br>
    * [3.3 Executing LLM response](#third-bullet-sub-3)<br>
4. [Proofs using Z3](#fourth-bullet)<br>
    * [4.1 Create necessary Z3 variables](#fourth-bullet-sub-1)<br>
    * [4.2 Define constraints and Z3 solver](#fourth-bullet-sub-2)<br>
5. [Saving to S3](#fifth-bullet)<br>
    * [Saving LLM response](#fifth-bullet-sub-1)<br>

The Notebook contains code snippets that recreates a Data Science / Appled Science user actions trying to access documents on the Onehouse data hub. Current demostration uses S3 bucket but in actuality it can be any database, S3 or other similar locations. The user action would call an LLM model of choice provided by Amazon Bedrock service. The user can use PDF and docx files and pass the data within these files to the supported LLM models like Anthropics Claude Opus / Sonnet / Haiku along with customized queries. The received responses can then be stored locally or in user specific S3 buckets as required.

### 1. Initial Setup <a class = "anchor" id = "first-bullet"></a>

#### 1.1 Installing Required Libraries <a class = "anchor" id = "first-bullet-sub-1"></a>

In [ ]:
!pip install pdfplumber
!pip install z3-solver

#### 1.2 Importing required libraries <a class = "anchor" id = "first-bullet-sub-2"></a>

In [ ]:
import boto3
import pdfplumber
import json
from botocore.exceptions import ClientError
import logging 
import io
import pandas as pd

### 2. Loading Data <a class = "anchor" id = "second-bullet"></a>


Downloading the specific pdf file from the user's S3 bucket. The user can browse through their S3 folder using the Amazon S3 console on this account. Accessing objects from any other S3 bucket results in an error. Modify the code in the cell to reflect the correct file name and the correct S3 bucket that the file is contained in.(There are currently no pdf samples)

In [ ]:
# Download pdf file from s3 bucket
session = boto3.Session()
s3 = session.client(service_name='s3')

s3.download_file('onehouse-project-ds-user-bucket',
                 'home/David/demo_ppa.pdf', 'demo.pdf')

The pdf file is then converted into a string and stored in a variable so that it can be passed in as a query to the LLM.

In [ ]:
# Open the PDF file
with pdfplumber.open('demo.pdf') as pdf:
    # Initialize an empty string to store the text
    fullText_pdf = ''

    # Loop through each page in the PDF
    for page in pdf.pages:
        # Extract the text from the page
        page_text = page.extract_text()

        # Append the page text to the overall text
        fullText_pdf += page_text + '\n'

### 3. LLM Prompting <a class = "anchor" id = "third-bullet"></a>

#### 3.1 Create model prompt function <a class = "anchor" id ="third-bullet-sub-1"></a>

The function below sends the provided content as a query to the specific LLM model provided by the user. Currently the only supported models are Anthropics Claude Opus/Haiku/Sonnet. The response from the model is returned as a string.

In [ ]:
# function that sends the request to the Claude model you want to send to, 
# requires the user to pass in the "content" portion of the body of the request
def prompt_model(model_type: str, content: str):
    if model_type == "CLAUDE_OPUS":
        model = "anthropic.claude-3-opus-20240229-v1:0" # Model ID does not work for some reason
    elif model_type == "CLAUDE_SONNET":
        model = "anthropic.claude-3-sonnet-20240229-v1:0"
    elif model_type == "CLAUDE_HAIKU":
        model = "anthropic.claude-3-haiku-20240307-v1:0"
    else: 
        print("Model ID not recognized")

    # Create a Bedrock Runtime client in the AWS Region of your choice.
    client = boto3.client("bedrock-runtime", region_name="us-east-1")

    # Set the model ID, e.g., Claude 3 Haiku.
    model_id = model

    # Format the request payload using the model's native structure. 
    native_request = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 10000,
        "temperature": 0.5,
        "messages": [
            {
                "role": "user",
                "content": json.loads(content),
            }
        ],
    }

    # Convert the native request to JSON.
    request = json.dumps(native_request)

    # Invoke the model with the request.
    streaming_response = client.invoke_model_with_response_stream(
        modelId=model_id, body=request
    )
    response_text =""
    # Extract and print the response text in real-time.
    for event in streaming_response["body"]:
        chunk = json.loads(event["chunk"]["bytes"])
        if chunk["type"] == "content_block_delta":
            response_text+=chunk["delta"].get("text", "")
    return response_text

#### 3.2 Prompting LLM <a class = "anchor" id = "third-bullet-sub-2"></a>

The function above can be called using an example similar to the one below. The content has to be provided in json format and the pdf document stored in the variable fullText_pdf is stored can be passed to the LLM through the query.

In [ ]:
# Sending prompt to model
prompt = "Create two strings start_date and end_date by writing code in python which correspond to the start and end of the discount term in the format YYYY-MM-DD."
response = prompt_model("CLAUDE_SONNET", json.dumps([{"type": "text", "text":fullText_pdf},
                                         {"type": "text", "text": prompt}]))
print(response)

#### 3.3 Executing LLM response <a class = "anchor" id = "third-bullet-sub-3"></a>

The cell below runs the python code generated by the LLM. It also stores all the variables created in the code that has been executed in the namespace 'variables'. Sometimes the generated response highlights the python code by including thriple ticks, otherwise it produces only the code as a response. The cell below accounts for both possibilities. If the LLM is requested to create the data and the user plans on using the data on their own after the LLM generated response is executed, it is important that they specify the name of the variable the data should be stored in so that they can access it. 

In [ ]:
# Find the start and end of the code block
start = response.find('```python')
end = response.find('```', start + 9)

if start == -1 or end == -1:
    code = response
    namespace = {}
    # Run the codes
    try:
        exec(code, namespace)
        # Find all the variables in the namespace
        variables = {k: v for k, v in namespace.items() if not k.startswith('__')}
        print("Code executed successfully.")
    except Exception as e:
        print("Error executing the code:"+str(e))
else:
    # Extract the code from the string
    code = response[start+9:end]
    namespace = {}
    # Run the codes
    try:
        exec(code, namespace)
        # Find all the variables in the namespace
        variables = {k: v for k, v in namespace.items() if not k.startswith('__')}
        print("Code executed successfully.")
    except Exception as e:
        print("Error executing the code:"+str(e))

In [ ]:
print(variables)

### 4. Proofs using Z3 <a class = "anchor" id = "fourth-bullet"></a>

The Z3 library is a powerful theorem prover and constraint solver developed by Microsoft Research. It can be used to solve a wide range of problems, including program verification, software testing, and logical reasoning. In this demonstration we will be using Z3 to create a simple consistency check in order to prove that the discount term end date is later than the discount term start date.

#### 4.1 Create necessary Z3 variables <a class = "anchor" id = "fourth-bullet-sub-1"></a>

In order to use the z3 solver, we will be counting the number of days that have been passed since a reference date. The dates that were provided as strings need to be converted into Z3 integer variables.


In [ ]:
from z3 import *
from datetime import datetime, date


reference_date = date(1970, 1, 1)
start_date = Int('start_date')
end_date = Int('end_date')

start_date_obj = datetime.strptime(variables['start_date'], '%Y-%m-%d').date()
end_date_obj = datetime.strptime(variables['end_date'], '%Y-%m-%d').date()

start_date_days = (start_date_obj - reference_date).days
end_date_days = (end_date_obj - reference_date).days

#### 4.2 Define constraints and Z3 solver <a class = "anchor" id = "fourth-bullet-sub-2"></a>

We define a constraint that the start date is less than the end date and add it to a Z3 Solver. You can add multiple constraints to a Solver and check if they are all true.

In [ ]:
constraints = [start_date < end_date] 

solver = Solver() 
solver.add(constraints) 
if solver.check() == sat:
    print("The discount term dates are in logical order.") 
else:
    print("The discount term dates are not in logical order")

### 5. Saving to S3 <a class = "anchor" id = "fifth-bullet"></a>

#### 5.1 Saving LLM response <a class = "anchor" id = "fifth-bullet-sub-1"></a>

If necessary the response from the LLM can be stored in a text file and then saved in the user specific S3 bucket.

In [ ]:
# Save response to a text file
textFile = open("response.txt","w")  # Change to 'a' if you do not want to overwrite
textFile.write(response)
textFile.close()
# Upload response text file to S3 bucket
s3 = boto3.resource('s3') 
s3.Bucket('onehouse-project-ds-user-bucket').upload_file("response.txt",'home/David/llm_response.txt')